<h1><img align="right" width="350" src="img/ATU-Logo-Full-RGB-Green.jpg"> Programming for Data Analysis - Project 1
</h1>
<p> 
Course: HDip in Computing in Data Analytics <br>
Module: Programming for Data Analysis <br>
Lecturer: Brian McGinley <br>
Project: Project 1 for the Programming for Data Analysis module of the HDip in Data Analytics beginning January 2023. 
    
Student: Eilis Donohue (G00006088)

Project spec:
See ProgDAProject.pdf

Software Used: 
 - Python v3.10 and higher
 - Jupyter Notebook 6.5.2   
 </p>

## Table of Contents
- [Introduction](#Intro)
- [Dataset Ingest and Preparation](#Prep)
- [Preliminary Analysis](#Prelim)
- [Overall Variable Correlation](#Corr)
- [Rainfall Analysis](#Rain)
- [Pressure Analysis](#Pressure)
- [Temperature Analysis](#Temperature)
- [Wind Speed Analysis](#Wind)
- [Data Synthesisation](#Synth)
- [Conclusions](#Conclusions)
- [References](#Refs)

## Introduction <a id="Intro"></a> <span style="font-size: 8pt;"> [[TOC]](#toc)</span>

The historical meteorological data from the weather station at Athenry, Co. Galway has been downloaded for 10 years from the met.ie website [1]. This dataset comprises a number of meteological variables as previewed below. The raw dataset (hly1875.csv) and its metadata is stored in the \data\Athenry_met folder. For the purposes of this study, the following variables have been chosen for analysis and data synthesisation.

- Rainfall
- Air Temperature
- Mean sealevel pressure
- Mean Wind speed

Each row of the raw dataset is an hourly recording of each variable.  This study is summarised as follows:

1. A preliminary analysis of the dataset (4 variables above) is performed to gain some insight into the overall trends and data distribution.
2. To simplify the data and its synthesisation, a daily aggregation of the variables is performed based on mean (and daily summation in the case of rainfall).
3. As weather is seasonal, a season variable is assigned to the data to allow for potentially more accurate distribution fitting and data synthesisation.
4. Each of the four variables above is analysed and a distribution is fitted based on curve fitting to the real data histograms. For some distributions, **Scipy stats** and **curve_fit** is used to generate fitted distributions and their parameters. The most appropriate distribution for each dataset based on a visual check.
5. Data is synthesised (for 1 year) using appropriate **Numpy** random number generators models using the distribution features found in step 4. 
6. The synthesised data is outputted to a csv file (synthesised_met_data.csv) in /data.

## Date Ingest and Preparation  <a id="Prep"></a> <span style="font-size: 8pt;"> [[TOC]](#toc)</span>

This section has the code used to ingest the data csv file and do some preparation of the data (i.e., convert the numeric fields to floats and remove blankspaces). The following steps are taken to prepare the data:

1. The data is read in to a **Pandas** dataframe.
2. The 4 variables of interest are isolated and copied to another dataframe.
3. The timestamp in the dataset is converted to a datetime object. The data is indexed by datetime as appropriate.  
4. Weather is inherently seasonal, so a new variable 'season' is added to the dataframe based on the month (i.e., meteorological winter is December to February inclusive, and so on).
5. The data is reduced to a daily aggregation based on the daily mean or sum (in the case of rainfall).

In [ ]:
# Import the required packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import norm, rayleigh, lognorm, multivariate_normal, expon, weibull_min
from scipy.optimize import *
import numpy as np
import math

In [ ]:
# Some function definitions
# Returns a season designation based on month to a dataframe 
def split_seasons(df_to_split, months):
  season_df = pd.DataFrame()
  for month in months:
    month_loc = df_to_split['datetime'].dt.month.between(month, month)
    season_df = pd.concat([season_df, df_to_split.loc[month_loc]])
  return season_df

# Returns Pearsons correlation on dataframe
def get_corr(data):
    return data.corr()

# assigns a season class to the data to allow for referencing of data without splitting dataframe
def assign_season_class(the_data, the_class, months):
  for month in months:
    month_loc = the_data['datetime'].dt.month.between(month, month)
    the_data.loc[month_loc, 'season'] = the_class
  return the_data

# function to find the midpoints of a array of bins (returned from plt.hist())
def mean_bins(given_bins):
  bins = []
  for i in range(len(given_bins[:-1])):
    bins.append((given_bins[i]+given_bins[i+1])/2)
  return np.array(bins)

# Define a styler for tables 
def my_styler(df, precision, caption):
    df_styler = df.style.format(precision=precision).set_caption(caption)
    return df_styler 

# Highlights values in a table between left and right values. Returns a styler object that can be displayed
def styler_highlight_between(df_styler, left, right):
    print(f"{left} {right}")
    df_styler = df_styler.highlight_between(color="#fffd75", left=left, right=right, inclusive='both') 
    return df_styler
  

In [ ]:
# Import the full met.ie dataset (downloaded from https://www.met.ie/climate/available-data/historical-data)
all_data_df = pd.read_csv("data/Athenry_met/hly1875.csv", skiprows=17)

In [ ]:
# Return all the columns of the raw dataset
all_data_df.columns.values

In [ ]:
# Set index to be datetime
all_data_df['datetime'] = pd.to_datetime(all_data_df['date'])
all_data_df = all_data_df.set_index(all_data_df['datetime'])
all_data_df.drop(columns=['date'], inplace=True)

In [ ]:
# Taking the data since 2012 to end 2022
time_start = "2012-01-01 00:00:00"
time_end = "2022-12-31 23:00:00"
# data_df is the time period to analyse
data_df = all_data_df.loc[time_start : time_end].copy()
data_df

In [ ]:
# Replace blankspace with nan [2]
data_df = data_df.replace(r'^\s+$', np.nan, regex=True)

In [ ]:
# Take 4 met variables - hourly rainfall, temperature, sealevel pressure and wind speed
# Convert the 4 chosen variables to float types
data_df[['rain', 'temp', 'msl', 'wdsp']] = data_df[['rain', 'temp', 'msl', 'wdsp']].astype(float)
# Create a smaller dataframe with the required variables
mydata_df = data_df[['rain', 'temp', 'msl', 'wdsp', 'datetime']].copy()


mydata_df_i = mydata_df.copy()
winter_months = [12, 1, 2]
spring_months = [3, 4, 5]
summer_months = [6, 7, 8]
autumn_months = [9, 10, 11]

# assign a new variable "season" to the data
mydata_df_i['season'] = ''
mydata_df_i = assign_season_class(mydata_df_i, 'winter', winter_months)
mydata_df_i = assign_season_class(mydata_df_i, 'spring', spring_months)
mydata_df_i = assign_season_class(mydata_df_i, 'summer', summer_months)
mydata_df_i = assign_season_class(mydata_df_i, 'autumn', autumn_months)
print(mydata_df_i.head(10))

In [ ]:


# Create a pairplot of the data for 3 years
sns.set_theme(style='whitegrid')
# take 3 years of data for the plot
data_3years_h = mydata_df_i.loc[mydata_df_i['datetime'].between('2020-01-01', '2020-12-31')]
g = sns.pairplot(data_3years_h, hue="season", diag_kind="kde")
g.map_lower(sns.kdeplot, levels=7, color=".2")
plt.show()

In [ ]:
# Display the correlation table for the entire dataset
corr_styler_list = []
df_corr =  get_corr(mydata_df_i.drop(columns = ["season", "datetime"]))    
df_styler = my_styler(df_corr, 2,  "Correlation - all data")
df_styler = styler_highlight_between(df_styler, 0.3, 0.99)
df_styler = styler_highlight_between(df_styler, -0.99, -0.3)
corr_styler_list = [df_styler]
class_names = ['winter', 'spring', 'summer', 'autumn']  # to allow looping over seasons

# Get correlation matrices for the 4 seasons and display
for i, item in enumerate(class_names):
    # Extract the data related to one season
    season_data = mydata_df_i[mydata_df_i["season"] == item].copy()
    # Strip the class column before passing to function
    season_data.drop(columns = ["season", "datetime"], inplace=True) 
    df_corr =  get_corr(season_data)      
    df_styler = my_styler(df_corr, 2, item + " Correlation")
    df_styler = styler_highlight_between(df_styler, 0.3, 0.99)
    df_styler = styler_highlight_between(df_styler, -0.99, -0.3)
    corr_styler_list.append(df_styler)
    
for item in corr_styler_list:
    display(item)


In [ ]:
# Aggregate the data based on daily mean or sum (in the case of rainfall)
mydata_agg = pd.DataFrame()
mydata_agg['Rainfall sum'] = data_df[['rain', 'datetime']].groupby(pd.Grouper(key='datetime', freq='D')).sum()
mydata_agg['Temp mean'] = data_df[['temp', 'datetime']].groupby(pd.Grouper(key='datetime', freq='D')).mean()
mydata_agg['Pressure mean'] = data_df[['msl', 'datetime']].groupby(pd.Grouper(key='datetime', freq='D')).mean()
mydata_agg['Windspeed mean'] = data_df[['wdsp', 'datetime']].groupby(pd.Grouper(key='datetime', freq='D')).mean()

In [ ]:
# Call a function to split the data and return the data based on a season
# Put datetime back as a variable [3]

mydata_agg_i = mydata_agg.reset_index()
winter_months = [12, 1, 2]
spring_months = [3, 4, 5]
summer_months = [6, 7, 8]
autumn_months = [9, 10, 11]

# assign a new variable "season" to the data
mydata_agg_i['season'] = ''
mydata_agg_i = assign_season_class(mydata_agg_i, 'winter', winter_months)
mydata_agg_i = assign_season_class(mydata_agg_i, 'spring', spring_months)
mydata_agg_i = assign_season_class(mydata_agg_i, 'summer', summer_months)
mydata_agg_i = assign_season_class(mydata_agg_i, 'autumn', autumn_months)
print(mydata_agg_i.head(10))

class_names = ['winter', 'spring', 'summer', 'autumn']  # to allow looping over seasons
var_names = ['Rainfall sum', 'Temp mean', 'Pressure mean', 'Windspeed mean'] # to allow looping over variables
plot_titles = ['Rainfall', 'Temperature', 'Pressure', 'Wind Speed']
units_names = ['(mm)', '(deg C)', '(hPa)', '(knots)']

## Preliminary Analysis <a id="Prelim"></a> <span style="font-size: 8pt;"> [[TOC]](#toc)</span> 

The preliminary analysis of the dataset is carried out to assess the overall statistics and variations of the variables.
 
- Firstly, the histograms of the entire dataset are plotted - this is the distribution of the hourly data. The number of bins was chosen based on iterative process to achieve best balance to minimise noise in the data, and to identify the underlying trends in the data.
- The boxplots are used to visualise the main summary statistics of the dataset. The median of each variable is represented by the orange line with the box itself enclosing the middle 50% of the data or interquartile range (from the 25% to the 75% quartile). Seasonal variations can be identified easily. The skew and spread of the data can be qualitatively shown.

In [ ]:
# Plots of the 8 year dataset for the 4 chosen variables
fig, axs = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle("Hourly met data at Athenry (2015-2022 inc)", fontsize=18)
axs[0,0].hist(data_df['rain'], bins=100)
axs[0,0].set_xlabel('Rainfall (mm)', fontsize=12)
axs[0,1].hist(data_df['temp'], bins=50)
axs[0,1].set_xlabel('Temperature (deg)', fontsize=12)
axs[1,0].hist(data_df['msl'], bins=50)
axs[1,0].set_xlabel('Sealevel Pressure (hPa)', fontsize=12)
axs[1,1].hist(data_df['wdsp'], bins=12)
axs[1,1].set_xlabel('Wind speed (knots)', fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
# Set up 1 fig with 4 boxplots
fig, axs = plt.subplots(2,2, figsize=(10, 8))
for i, var in enumerate(var_names):
    ax_list = [(0,0), (0,1), (1,0), (1,1)] 

    # create list of data to do a boxplot
    plot_data = [mydata_agg_i[var].loc[mydata_agg_i['season'] == 'winter'],
                mydata_agg_i[var].loc[mydata_agg_i['season'] == 'spring'],
                mydata_agg_i[var].loc[mydata_agg_i['season'] == 'summer'],
                mydata_agg_i[var].loc[mydata_agg_i['season'] == 'autumn']]

    # whis is arbitrarily high so that the whiskers represent min and max
    axs[ax_list[i]].boxplot(plot_data, whis=10000)   
    axs[ax_list[i]].set_ylabel(f'{var} {units_names[i]}')
    axs[ax_list[i]].set_xticklabels(class_names, fontsize=10)
    axs[ax_list[i]].set_title(plot_titles[i], fontsize=12)
    axs[ax_list[i]].grid(visible=True, which='both', axis='both')
fig.tight_layout() 


In [ ]:
# troubleshooting pressure for autumn (for some reason, the boxplot will not display for pressure in autumn)

plot_data = [mydata_agg_i['Pressure mean'].loc[mydata_agg_i['season'] == 'winter'],
                mydata_agg_i['Pressure mean'].loc[mydata_agg_i['season'] == 'spring'],
                mydata_agg_i['Pressure mean'].loc[mydata_agg_i['season'] == 'summer'],
                mydata_agg_i['Pressure mean'].loc[mydata_agg_i['season'] == 'autumn']]

print(mydata_agg_i['Pressure mean'].loc[mydata_agg_i['season'] == 'autumn'].describe())

In [ ]:
# Create a pairplot of the data for 3 years
sns.set_theme(style='whitegrid')
# take 3 years of data for the plot
data_3years = mydata_agg_i.loc[mydata_agg_i['datetime'].between('2020-01-01', '2022-12-31')]
g = sns.pairplot(data_3years, hue="season", diag_kind="kde")
g.map_lower(sns.kdeplot, levels=7, color=".2")
plt.show()

## Overall Variable Correlation <a id="Corr"></a> <span style="font-size: 8pt;"> [[TOC]](#toc)</span> 
In order to investigate correlation between the data variables, the Pearson correlation method in pandas is applied to the data (by season).

- Overall, there is no very strong correlation between any of the data variables investigated as part of this study. This was inferred above in the pairplot. Correlation values greater or less than 0.3 and -0.3 respectively are highlighted in the tables below.  

- The strongest correlations are between pressure and windspeed and pressure and rainfall. The correlation between pressure and windspeed ranges between -0.37 and -0.46 and ranges from from -0.49 to -0.42 between pressure and rainfall.   

- Winter shows the highest degree of correlation between the variables.

In [ ]:
# Display the correlation table for the entire dataset
corr_styler_list = []
df_corr =  get_corr(mydata_agg_i.drop(columns = ["season", "datetime"]))    
df_styler = my_styler(df_corr, 2,  "Correlation - all data")
df_styler = styler_highlight_between(df_styler, 0.3, 0.99)
df_styler = styler_highlight_between(df_styler, -0.99, -0.3)
corr_styler_list = [df_styler]

# Get correlation matrices for the 4 seasons and display
for i, item in enumerate(class_names):
    # Extract the data related to one season
    season_data = mydata_agg_i[mydata_agg_i["season"] == item].copy()
    # Strip the class column before passing to function
    season_data.drop(columns = ["season", "datetime"], inplace=True) 
    df_corr =  get_corr(season_data)      
    df_styler = my_styler(df_corr, 2, item + " Correlation")
    df_styler = styler_highlight_between(df_styler, 0.3, 0.99)
    df_styler = styler_highlight_between(df_styler, -0.99, -0.3)
    corr_styler_list.append(df_styler)
    
for item in corr_styler_list:
    display(item)



## Rainfall Analysis <a id="Rain"></a> <span style="font-size: 8pt;"> [[TOC]](#toc)</span> 
### Rainfall data analysis
- The rainfall is highly skewed towards values of zero. The data here may be seen as split into two regimes - the time or days when there is no rain and then those times or days when there is rain as is done here [4].
- It is useful to remove the 0 rainfall days by imposing a 1mm limit to determine a "rainy day" v "dry day". This approach is used here [4].
- Rainfall over a period of time can follow an exponential distribution [4] when the 0 rainfall periods are removed. The exponential distribution is shown below and appears to be a good fit to the "rainy day" data [6].  

The exponential distribution:

$$
f(x;\lambda) = 
\begin{cases} 
\lambda e^{-\lambda x} & \text{if } x \geq 0 \\
0 & \text{if } x < 0 
\end{cases}
$$

In [ ]:
# Do some analysis on the daily aggregated rainfall data over 10 years.
rainfall_df = mydata_agg_i[['Rainfall sum', 'datetime', 'season']]
rainfall_df = rainfall_df.set_index('datetime')
max_rain = rainfall_df['Rainfall sum'].max()
min_rain = rainfall_df['Rainfall sum'].min()

fig, axs = plt.subplots(2,2, figsize=(10, 10));
# list of axes for looping
ax_list = [(0,0), (0,1), (1,0), (1,1)] 
fig.suptitle('Rainfall data normalised distribution with exponential curve fit', fontsize=16)

# Define empty lists to store the rainfall params and frequency from the curve fit for data synthesisation later
rainfall_exp_params_season = []
fw_seasonal = []
for i, season in enumerate(class_names):
  rainfall_df_season = pd.DataFrame()
  rainfall_df_season = rainfall_df[rainfall_df['season'] == season].drop(columns=['season'])
  # frequency of rainy days with criterion of >1mm rainfall in a day
  fw = (rainfall_df_season['Rainfall sum']>1).sum() / len(rainfall_df_season)
  fw_seasonal.append(fw)
  daily_rain_rainydays = rainfall_df_season[rainfall_df_season['Rainfall sum']>1]
  # defining the bin size
  bin_size = 2
  # find the min and max of the data
  daily_rain_rainydays_floor = math.floor(daily_rain_rainydays['Rainfall sum'].min())
  daily_rain_rainydays_ceiling = math.ceil(daily_rain_rainydays['Rainfall sum'].max())

  # generate a list of bins based on the above to fit the data
  bins = np.arange(daily_rain_rainydays_floor, daily_rain_rainydays_ceiling+bin_size, bin_size)

  # returns the normalised data for the histogram (area under curve =1)

  n, bins_returned, _ = axs[ax_list[i]].hist(daily_rain_rainydays['Rainfall sum'], bins=bins, density=True)
  bin_start = np.array(bins[:-1])

  # Use curve_fit method as in: https://stackoverflow.com/questions/50448199/lognormal-curve-fit
  # PDF for the exponential distribution - so that curve fitting can be done
  def f_exp(bin_start, lamda_1):
    return lamda_1*np.exp(-1*lamda_1*bin_start)

  # returns the exponential distribution parameters for the data fit
  params, extras = curve_fit(f_exp, bin_start, n)

  # fit a exponential curve
  lamda_1 = params[0]
  pdf_expon = (lamda_1*np.exp(-1*lamda_1*bin_start))
  rainfall_exp_params_season.append(lamda_1)

  axs[ax_list[i]].plot(bin_start, pdf_expon, 'r', label='Exponential PDF')
  axs[ax_list[i]].set_xlabel(f'Daily Rainfall Sum (mm)', fontsize=10)
  axs[ax_list[i]].set_ylabel(f'Normalised histogram', fontsize=10)
  axs[ax_list[i]].set_title(season, fontsize=14)
  axs[ax_list[i]].grid(visible=True, which='both', axis='both')
axs[(0,0)].legend();
fig.tight_layout()
fig, ax = plt.subplots(1, figsize=(3, 4));
ax.bar(class_names, fw_seasonal);
ax.set_title("Frequency of rainy days (fw)");


In [ ]:
# Get a linear regression plot between rainfall (>1mm) and pressure 
fig, axs = plt.subplots(1,2, figsize=(12, 5))

axs[0].scatter(mydata_agg_i['Pressure mean'].loc[mydata_agg_i['Rainfall sum']>1], mydata_agg_i['Rainfall sum'].loc[mydata_agg_i['Rainfall sum']>1]);
res = stats.linregress(mydata_agg_i['Pressure mean'].loc[mydata_agg_i['Rainfall sum']>1], mydata_agg_i['Rainfall sum'].loc[mydata_agg_i['Rainfall sum']>1], alternative='two-sided')
print(res)
axs[0].plot(mydata_agg_i['Pressure mean'].loc[mydata_agg_i['Rainfall sum']>1], res.intercept + res.slope*mydata_agg_i['Pressure mean'].loc[mydata_agg_i['Rainfall sum']>1], 'r', label='linear fitted line')
axs[0].set_ylabel("Daily Rainfall (mm)")
axs[0].set_xlabel("Mean Pressure (hPa)")
axs[0].legend()

#r = np.corrcoef(mydata_agg_i['Pressure mean'].loc[mydata_agg_i['Rainfall sum']>1], mydata_agg_i['Rainfall sum'].loc[mydata_agg_i['Rainfall sum']>1])
#print(r)

# Get a linear regression plot between log(rainfall) (>1mm) and pressure 

res_log = stats.linregress(mydata_agg_i['Pressure mean'].loc[mydata_agg_i['Rainfall sum']>1], np.log(mydata_agg_i['Rainfall sum'].loc[mydata_agg_i['Rainfall sum']>1]), alternative='two-sided')
print(f'Log linear fit {res_log}')
axs[1].scatter(mydata_agg_i['Pressure mean'].loc[mydata_agg_i['Rainfall sum']>1], np.log(mydata_agg_i['Rainfall sum'].loc[mydata_agg_i['Rainfall sum']>1]));
axs[1].plot(mydata_agg_i['Pressure mean'].loc[mydata_agg_i['Rainfall sum']>1], res_log.intercept + res_log.slope*mydata_agg_i['Pressure mean'].loc[mydata_agg_i['Rainfall sum']>1], 'r', label='loglinear fitted line')
axs[1].set_ylabel("Log(Daily Rainfall) (mm)")
axs[1].set_xlabel("Mean Pressure (hPa)")
axs[1].legend()
fig.tight_layout()
plt.show()

In [ ]:
# Get a linear regression plot between windspeed and pressure 
fig, axs = plt.subplots(2,2, figsize=(12, 12))
ax_list = [(0,0), (0,1), (1,0), (1,1)] 

# Define empty lists to store the seasonal linear regression results to a list
wind_pressure_linregress_season = []

for i, season in enumerate(class_names):

#mydata_agg_i[var].loc[mydata_agg_i['season'] == 'winter'
  axs[ax_list[i]].scatter(mydata_agg_i['Pressure mean'].loc[mydata_agg_i['season'] == season], mydata_agg_i['Windspeed mean'].loc[mydata_agg_i['season'] == season])
  res_ws = stats.linregress(mydata_agg_i['Pressure mean'].loc[mydata_agg_i['season'] == season], mydata_agg_i['Windspeed mean'].loc[mydata_agg_i['season'] == season])
  wind_pressure_linregress_season.append((res_ws.slope, res_ws.intercept))
  
  print(res_ws)

  axs[ax_list[i]].plot(mydata_agg_i['Pressure mean'], res_ws.intercept + res_ws.slope*mydata_agg_i['Pressure mean'], 'r', label='linear fitted line')
  axs[ax_list[i]].set_ylabel("Mean windspeed (Kts)")
  axs[ax_list[i]].set_xlabel("Mean Pressure (hPa)")
  axs[ax_list[i]].legend()


fig.tight_layout()
plt.show()

## Pressure Analysis <a id="Pressure"></a> <span style="font-size: 8pt;"> [[TOC]](#toc)</span> 
The seasonal mean sea level pressure is plotted below and a log normal distribution is fitted to the data.

- From the preliminary analysis above, the distribution of pressure is skewed to the left.
- Trying a lognormal normal distribution seems to provide a reasonably accurate fit to the data. There is some underestimation at lower values. 


In [ ]:
# Define empty lists to store the pressure params and frequency from the curve fit for data synthesisation later
pressure_params_season = []
pressure_df = mydata_agg_i[['Pressure mean', 'datetime', 'season']]
pressure_df = pressure_df.set_index('datetime')
# 
fig, axs = plt.subplots(2,2, figsize=(10, 10));
fig.suptitle('Rainfall data with lognormal curve fit', fontsize=16)
# Axis list for looping over axes below
ax_list = [(0,0), (0,1), (1,0), (1,1)] 
  
for i, season in enumerate(class_names):
  pressure_df_season = pd.DataFrame()
  pressure_df_season = pressure_df[pressure_df['season'] == season].drop(columns=['season'])
  
  # defining the bin size
  bin_size = 5
  # find the min and max of the data
  data_floor = math.floor(pressure_df_season['Pressure mean'].min())
  data_ceiling = math.ceil(pressure_df_season['Pressure mean'].max())

  # Generate a list of bins based on the min and max above to fit the data
  bins = np.arange(data_floor, data_ceiling+bin_size, bin_size)
  # Returns the normalised data for the histogram (area under curve =1)
  n, bins_returned, _ = axs[ax_list[i]].hist(pressure_df_season['Pressure mean'], bins=bins, density=True)
  
  # Gets the mid-point of the bins
  bin_mid = mean_bins(bins_returned)

# https://en.wikipedia.org/wiki/Log-normal_distribution
# This could be moved into functions section 
  def f_lognormal(x, mu, sigma):
    return 1/(np.sqrt(2*np.pi)*sigma*x)*np.exp(-((np.log(x)-mu)**2)/(2*sigma**2))

  # returns mu and sigma in params 
  params, extras = curve_fit(f_lognormal, bin_mid, n)
  mu_pressure = params[0]
  sigma_pressure = params[1]
  # store tuples to this list for use in the data synth later
  pressure_params_season.append((mu_pressure, sigma_pressure))
  # fit a lognormal curve
  pdf_lognormal = f_lognormal(bin_mid, mu_pressure, sigma_pressure)

  axs[ax_list[i]].plot(bin_mid, pdf_lognormal, 'r', label='Lognormal PDF')
  axs[ax_list[i]].set_xlabel(f'Daily Mean Pressure (hPa)', fontsize=10)
  axs[ax_list[i]].set_ylabel(f'Normalised histogram', fontsize=10)
  axs[ax_list[i]].set_title(season, fontsize=14)
  axs[ax_list[i]].grid(visible=True, which='both', axis='both')
axs[(0,0)].legend();
fig.tight_layout()

## Temperature Data Analysis <a id="Temperature"></a> <span style="font-size: 8pt;"> [[TOC]](#toc)</span> 
Qualitatively, the preliminary analysis of temperature distribution above shows a broadly normal distribution.  The normal distribution PDF is given below.
A Log normal PDF curve was also fitted. The normal and log normal fits are almost coincident for winter, spring and autumn which shows that there is no significant skew in the data and therefore, a normal distribution is fitted. The data for summer however, shows a slight skew to the left, so the log normal fit parameters will be taken for summer. 

The normal distribution:

$$ f(x) = \frac{1}{\sigma \sqrt{2 \pi}} e^{-\frac{1}{2} \big( \frac{x- \mu}{\sigma} \big)^2} $$

In [ ]:
# Define empty lists to store the temperature params and frequency from the curve fit for data synthesisation later
temp_params_season_normal = []
temp_params_season_lognormal = []
temp_df = mydata_agg_i[['Temp mean', 'datetime', 'season']]
temp_df = temp_df.set_index('datetime')
# 
fig, axs = plt.subplots(2,2, figsize=(10, 10));
fig.suptitle('Temperature data normalised distribution with normal curve fit', fontsize=16)
# Axis list for looping over axes below
ax_list = [(0,0), (0,1), (1,0), (1,1)] 
  
for i, season in enumerate(class_names):
  temp_df_season = pd.DataFrame()
  temp_df_season = temp_df[temp_df['season'] == season].drop(columns=['season'])
  
  # defining the bin size
  bin_size = 1
  # find the min and max of the data
  data_floor = math.floor(temp_df_season['Temp mean'].min())
  data_ceiling = math.ceil(temp_df_season['Temp mean'].max())

  # Generate a list of bins based on the min and max above to fit the data
  bins = np.arange(data_floor, data_ceiling+bin_size, bin_size)
  # Returns the normalised data for the histogram (area under curve =1)
  n, bins_returned, _ = axs[ax_list[i]].hist(temp_df_season['Temp mean'], bins=bins, density=True)
  
  # Gets the mid-point of the bins
  bin_mid = mean_bins(bins_returned)
  mu_temp = temp_df_season['Temp mean'].mean()
  sigma = temp_df_season['Temp mean'].std()
  # Getting curve fits this time with scipy stats
  param_normal = norm.fit(temp_df_season['Temp mean'])
  print(param_normal)
  param_lognormal = lognorm.fit(temp_df_season['Temp mean'])
  print(param_lognormal)
  # plotting the fitted pdf curves for normal and log normal
  pdf_fitted_norm = norm.pdf(bin_mid, loc=param_normal[0], scale=param_normal[1])
  pdf_fitted_lognorm = lognorm.pdf(bin_mid, param_lognormal[0], param_lognormal[1], param_lognormal[2])
  # store tuples to this list for use in the data synth later
  temp_params_season_normal.append((param_normal[0], param_normal[1]))
  temp_params_season_lognormal.append((param_lognormal[0], param_lognormal[1], param_lognormal[2]))
  # fit a normal curve
  axs[ax_list[i]].plot(bin_mid, pdf_fitted_norm, 'r', label='Normal PDF')
  axs[ax_list[i]].plot(bin_mid, pdf_fitted_lognorm, 'g', label='Lognormal PDF')
  
  axs[ax_list[i]].set_xlabel(f'Daily Mean Temp (deg C)', fontsize=10)
  axs[ax_list[i]].set_ylabel(f'Normalised histogram', fontsize=10)
  axs[ax_list[i]].set_title(season, fontsize=14)
  axs[ax_list[i]].grid(visible=True, which='both', axis='both')
axs[(0,0)].legend();
fig.tight_layout()

## Wind Speed Data Analysis <a id="Wind"></a> <span style="font-size: 8pt;"> [[TOC]](#toc)</span> 

The preliminary analysis shows mean wind speed to have a tail for higher values. This distribution does not look normal or indeed log normal as the temperature and pressure above does.  A rayleigh distribution is often used to model wind speed, however, Weibull is also used [12] for modelling extreme values.

The data below seems well represented by a mixture of Log normal (summer and autumn) and Rayleigh (winter and spring). Generally, the log normal and Rayleigh distributions are a better fit to the data than the normal distribution which fails to capture the extreme values as accuractly as Rayleigh and Log normal.


In [ ]:
# Define empty lists to store the wind params and frequency from the curve fit for data synthesisation later
wind_params_season_normal = []
wind_params_season_lognormal = []
wind_params_season_rayleigh = []
# isolate the wind data
wind_df = mydata_agg_i[['Windspeed mean', 'datetime', 'season']]
wind_df = wind_df.set_index('datetime')
# 
fig, axs = plt.subplots(2,2, figsize=(10, 10));
fig.suptitle('Wind speed data normalised distribution with curve fits', fontsize=16)
# Axis list for looping over axes below
ax_list = [(0,0), (0,1), (1,0), (1,1)] 
  
for i, season in enumerate(class_names):
  wind_df_season = pd.DataFrame()
  wind_df_season = wind_df[wind_df['season'] == season].drop(columns=['season'])
  
  # defining the bin size
  bin_size = 1
  # find the min and max of the data
  data_floor = math.floor(wind_df_season['Windspeed mean'].min())
  data_ceiling = math.ceil(wind_df_season['Windspeed mean'].max())

  # Generate a list of bins based on the min and max above to fit the data
  bins = np.arange(data_floor, data_ceiling+bin_size, bin_size)
  # Returns the normalised data for the histogram (area under curve =1)
  n, bins_returned, _ = axs[ax_list[i]].hist(wind_df_season['Windspeed mean'], bins=bins, density=True)
  
  # Gets the mid-point of the bins
  bin_mid = mean_bins(bins_returned)
  mu_wind = wind_df_season['Windspeed mean'].mean()
  sigma = wind_df_season['Windspeed mean'].std()
  # Getting curve fits this time with scipy stats
  param_normal = norm.fit(wind_df_season['Windspeed mean'])
  param_lognormal = lognorm.fit(wind_df_season['Windspeed mean'])
  param_rayleigh = rayleigh.fit(wind_df_season['Windspeed mean'])
  print(param_lognormal)
  param_weibull = weibull_min.fit(wind_df_season['Windspeed mean'])
  # Generate some fitted curves using scipy stats 
  pdf_fitted_norm = norm.pdf(bin_mid, loc=param_normal[0], scale=param_normal[1])
  pdf_fitted_lognorm = lognorm.pdf(bin_mid, param_lognormal[0], param_lognormal[1], param_lognormal[2])
  pdf_fitted_rayleigh = rayleigh.pdf(bin_mid, param_rayleigh[0], param_rayleigh[1])
  pdf_fitted_weibull = weibull_min.pdf(bin_mid, param_weibull[0], param_weibull[1], param_weibull[2])
  # store tuples to this list for use in the data synth later
  wind_params_season_normal.append((param_normal[0], param_normal[1]))
  wind_params_season_lognormal.append((param_lognormal[0], param_lognormal[1], param_lognormal[2]))
  wind_params_season_rayleigh.append((param_rayleigh[0], param_rayleigh[1]))
  # plotting the fitted pdf curves for normal and log normal
  axs[ax_list[i]].plot(bin_mid, pdf_fitted_norm, 'r', label='Normal PDF')
  axs[ax_list[i]].plot(bin_mid, pdf_fitted_lognorm, 'g', label='Lognormal PDF')
  axs[ax_list[i]].plot(bin_mid, pdf_fitted_rayleigh, 'c', label='Rayleigh PDF')
  axs[ax_list[i]].plot(bin_mid, pdf_fitted_weibull, 'm', label='Weibull PDF')
  
  axs[ax_list[i]].set_xlabel(f'Daily Mean windspeed (Kts)', fontsize=10)
  axs[ax_list[i]].set_ylabel(f'Normalised histogram', fontsize=10)
  axs[ax_list[i]].set_title(season, fontsize=14)
  axs[ax_list[i]].grid(visible=True, which='both', axis='both')
axs[(0,0)].legend();
fig.tight_layout()

## Data Synthesisation <a id="Synth"></a> <span style="font-size: 8pt;"> [[TOC]](#toc)</span>

In the preceding sections, a collection of PDF fitted curves and associated parameters has been generated based on the dataset daily aggregation.  In this section, the synthesisation of the meteorological values will be attempted, using the fitted parameters for each of the 4 seasons and using Numpy random generators as appropriate.

Note that synthesised data will be indicated "_s" below.

### Rainfall Data Synthesisation
Daily rainfall data synthesisation will be carried out as follows:

The frequency of rainless days with a frequency of (1-fw) will be used to generate a corresponding number of days in the synthesised data with no rain. 
- The number of dry days is found by subtracting rainy days from the total number of days and a value of 0 is assigned.
- It is possible to generate data for 1 year to synthesise daily rainfall totals by the following:
    -- generate some random data based on the exponential distribution parameters which describe the rainfall daily total distribution. The size of the random data is based on the frequency of rainy days, fw, multiplied by the total number of days of required data [14]. 
- The 2 arrays are joined together and then shuffled randomly.

### Pressure Data
Pressure data was generated with the numpy.random.lognormal generator.

### Temperature Data
Temperature data was generated with a numpy.random.normal generator.

### Windspeed Data
Windspeed data was synthesised twice by two different methods.

1. Windspeed data was generated with Numpy Rayleigh random generator 
2. A seasonal linear regression fit was made on the real data between pressure and windspeed. This linear regression slope and intercept was then applied to the synthesised pressure data along with a tuned noise parameter to synthesis windspeed data.

    


In [ ]:
# First, set up some parameters for the synthesised dataset
# no_days is number of rows in synthesised dataset
no_years = 5
var_names_synth = ['Rainfall sum_s', 'Temp mean_s', 'Pressure mean_s', 'Windspeed mean_s', 'Windspeed linregress_s', 'season'] # to allow looping over variables
# empty dataframe for storing the synthesised data
# add column for day number https://stackoverflow.com/questions/16327055/how-to-add-an-empty-column-to-a-dataframe
# no of days in each season
season_days = [31+31+28, 31+30+31, 30+31+31, 30+31+30]
met_data_synth = pd.DataFrame(index=range(365*no_years), columns=var_names_synth)
print(met_data_synth)
# set up a pdf fit dictionary for temperature and wind
temp_fit_dict = {'winter':'normal', 'spring':'normal', 'summer':'normal', 'autumn':'normal'}
wind_fit_dict = {'winter':'rayleigh', 'spring':'rayleigh', 'summer':'rayleigh', 'autumn':'rayleigh'}

# Assign the seasons to the empty dataframe - in the correct order, tho it shouldn't matter because each year is generated identically
start_index = 0

for i in range(no_years):
  for j, season in enumerate(class_names):
    season_class = np.array(season_days[j] * [class_names[j]])
    end_index = start_index + len(season_class) -1 
    met_data_synth['season'].loc[start_index:end_index] = season_class
    start_index = end_index + 1


In [ ]:
# Rainfall synthesisation
# set up a random generator for exponential distribution
# https://numpy.org/devdocs/reference/random/generated/numpy.random.Generator.exponential.html
# generate 1 year of daily rainfall totals - assuming that the frequency of rainy days calculated above
start_index = 0
for y in range(no_years):
  # initialise a numpy array from an empty  list
  rainfall_syn_data = np.array([])
  for i, season in enumerate(class_names):
    # Generate the random rainfall data using the numpy exponential 
    random_rainfall_season = np.random.default_rng().exponential(1/rainfall_exp_params_season[i], int(season_days[i]*fw_seasonal[i]))
    # Get the number of rainy days and non-rainy days
    rainfall_days_season = int(fw_seasonal[i] * season_days[i])
    non_rainfall_days_season = np.array((season_days[i]-rainfall_days_season)*[0])
    rainfall_syn_data_season = np.concatenate((random_rainfall_season, non_rainfall_days_season))
    # Mix up the 0 rainfall and rainfall days randomly  
    np.random.shuffle(rainfall_syn_data_season)
    rainfall_syn_data = np.concatenate((rainfall_syn_data, rainfall_syn_data_season)) 
  end_index = start_index + 365 - 1 
  met_data_synth['Rainfall sum_s'].loc[start_index:end_index] = rainfall_syn_data
  start_index = end_index + 1

# compare the real data with the synthesised data
fig, ax = plt.subplots(1, figsize=(5, 5));
fig.suptitle('Rainfall data real v synthesised', fontsize=12)
ax.hist(mydata_agg_i['Rainfall sum'], bins=10, density=True, label='real')
ax.hist(met_data_synth['Rainfall sum_s'], bins=10, density=True, alpha=0.5, label='synthesised')
ax.set_xlabel(f'Rainfall (mm)', fontsize=12)
ax.set_ylabel(f'Normalised histogram', fontsize=12)
ax.legend()
fig.tight_layout()
plt.show()


In [ ]:
# Temperature synthesisation


start_index = 0
for y in range(no_years):
  # initialise a numpy array from an empty  list
  temp_syn_data = np.array([])
  for i, season in enumerate(class_names):
    # Generate the random temp data using the numpy normal or log normal generator as per dictionary defined above
    if temp_fit_dict[season] == "normal":
      random_temp_season = np.random.default_rng().normal(temp_params_season_normal[i][0], temp_params_season_normal[i][1], season_days[i])
    elif temp_fit_dict[season] == "lognormal":
      # slightly different as lognormal params were generated from scipy stats
      random_temp_season = np.random.lognormal(temp_params_season_lognormal[i][0], temp_params_season_lognormal[i][1], season_days[i])

    temp_syn_data = np.concatenate((temp_syn_data, random_temp_season))
  end_index = start_index + 365 - 1 
  met_data_synth['Temp mean_s'].loc[start_index:end_index] = temp_syn_data
  start_index = end_index + 1  

# compare the real data with the synthesised data
fig, ax = plt.subplots(1, figsize=(5, 5));
fig.suptitle('Temperature data real v synthesised', fontsize=12)
ax.hist(mydata_agg_i['Temp mean'], bins=15, density=True, label='real')
ax.hist(met_data_synth['Temp mean_s'], bins=15, density=True, alpha=0.5, label='synthesised')
ax.set_xlabel(f'Temp (deg C)', fontsize=12)
ax.set_ylabel(f'Normalised histogram', fontsize=12)
ax.legend()
fig.tight_layout()
plt.show()


In [ ]:
# Pressure data synthesisation

start_index = 0
for y in range(no_years):
  # initialise a numpy array from an empty list
  pressure_syn_data = np.array([])
  for i, season in enumerate(class_names):
    # Generate the random pressure data using the log normal generator 
    random_pressure_season = np.random.default_rng().lognormal(pressure_params_season[i][0], pressure_params_season[i][1], season_days[i])
    # add the season's data onto the end of the array
    pressure_syn_data = np.concatenate((pressure_syn_data, random_pressure_season))
  end_index = start_index + 365 - 1 
  met_data_synth['Pressure mean_s'].loc[start_index:end_index] = pressure_syn_data
  start_index = end_index + 1  
  
# compare the real data with the synthesised data
fig, ax = plt.subplots(1, figsize=(5, 5));
fig.suptitle('Pressure data real v synthesised', fontsize=12)
ax.hist(mydata_agg_i['Pressure mean'], bins=15, density=True, label='real')
ax.hist(met_data_synth['Pressure mean_s'], bins=15, density=True, alpha=0.5, label='synthesised')
ax.set_xlabel(f'Pressure (hPa)', fontsize=12)
ax.set_ylabel(f'Normalised histogram', fontsize=12)
ax.legend()
fig.tight_layout()
plt.show()


In [ ]:
# Wind speed synthesisation - changing to rayleigh for all as issues with lognormal
wind_fit_dict = {'winter':'rayleigh', 'spring':'rayleigh', 'summer':'rayleigh', 'autumn':'rayleigh'}
start_index = 0
for y in range(no_years):
  # initialise a numpy array from an empty list
  wind_syn_data = np.array([])
  for i, season in enumerate(class_names):
    random_wind_season = np.array([])
    # Generate the random temp data using the numpy normal or log normal generator as per dictionary defined above
    if wind_fit_dict[season] == "lognormal":
      mean = np.log(wind_params_season_lognormal[i][2])
      #print(mean)
      std_dev = wind_params_season_lognormal[i][0]
      #print(std_dev)
      random_wind_season = np.random.default_rng().lognormal(wind_params_season_lognormal[i][0], wind_params_season_lognormal[i][1], season_days[i])
      #random_wind_season = np.random.default_rng().lognormal(mean, std_dev, season_days[i])
      #print(random_wind_season)
    elif wind_fit_dict[season] == "rayleigh":
      mode = np.sqrt(2 / np.pi) * wind_params_season_rayleigh[i][1]
      mode = mode +1
      random_wind_season = np.random.rayleigh(mode, season_days[i])
      #print(wind_params_season_rayleigh[i][0])
    wind_syn_data = np.concatenate((wind_syn_data, random_wind_season))
    end_index = start_index + 365 - 1 
  met_data_synth['Windspeed mean_s'].loc[start_index:end_index] = wind_syn_data
  start_index = end_index + 1  

# compare the real data with the synthesised data
fig, ax = plt.subplots(1, figsize=(5, 5));
fig.suptitle('Wind speed data real v synthesised', fontsize=12)
ax.hist(mydata_agg_i['Windspeed mean'], bins=15, density=True, label='real')
ax.hist(met_data_synth['Windspeed mean_s'], bins=15, density=True, alpha=0.5, label='synthesised')
ax.set_xlabel(f'Windspeed (Kts)', fontsize=12)
ax.set_ylabel(f'Normalised histogram', fontsize=12)
ax.legend()
fig.tight_layout()
plt.show()
# Finally write out the synthesised dataset to a csv file
met_data_synth.to_csv("data/synthesised_met_data.csv")


In [ ]:
# Wind speed synthesisation - based on linear regression fit from above

#wind_pressure_linregress_season
start_index = 0
# initialise a numpy array from an empty list
#wind_syn_data = np.array([])

fig, axs = plt.subplots(2,2, figsize=(10, 10));
fig.suptitle('Wind speed data synthesised with lin regress results', fontsize=16)
# Axis list for looping over axes below
ax_list = [(0,0), (0,1), (1,0), (1,1)] 
# tuned noise parameters for each season
noise_season = [[10, 5], [9.5,4.5], [7,3.5], [6.2, 2.9]]
for i, season in enumerate(class_names):
  random_wind_season = np.array([])
  # Generate the random temp data using the numpy normal or log normal generator as per dictionary defined above
  rng=np.random.default_rng()
  # Generate some random noise
  noise = ((rng.random(len(met_data_synth['Pressure mean_s'].loc[met_data_synth['season']==season]))*noise_season[i][0])-noise_season[i][1])
  # model autumn on summer (no linear regression data for autumn)
  if season == "autumn":
    wind_pressure_linregress_season[3] = wind_pressure_linregress_season[2][0], wind_pressure_linregress_season[2][1]
    
  met_data_synth['Windspeed linregress_s'].loc[met_data_synth['season']==season] = (met_data_synth['Pressure mean_s'].loc[met_data_synth['season']==season] * wind_pressure_linregress_season[i][0]) + wind_pressure_linregress_season[i][1] + noise
  axs[ax_list[i]].hist(mydata_agg_i['Windspeed mean'].loc[mydata_agg_i['season']==season], density=True, label='real')
  axs[ax_list[i]].hist(met_data_synth['Windspeed linregress_s'].loc[met_data_synth['season']==season], density=True, alpha=0.5, label='synthesised')   
  axs[ax_list[i]].set_xlabel(f'Daily Mean windspeed (Kts)', fontsize=10)
  axs[ax_list[i]].set_ylabel(f'Normalised histogram', fontsize=10)
  axs[ax_list[i]].set_title(season, fontsize=14)

# compare the real data with the synthesised data
fig, ax = plt.subplots(1, figsize=(5, 5));
fig.suptitle('Wind speed data real v synthesised', fontsize=12)
ax.hist(mydata_agg_i['Windspeed mean'], bins=12, density=True, label='real')
ax.hist(met_data_synth['Windspeed linregress_s'], bins=12, density=True, alpha=0.5, label='synthesised')
ax.set_xlabel(f'Windspeed (Kts)', fontsize=12)
ax.set_ylabel(f'Normalised histogram', fontsize=12)
ax.legend()
fig.tight_layout()
plt.show()
# Finally write out the synthesised dataset to a csv file
met_data_synth.to_csv("data/synthesised_met_data.csv")


In [ ]:
# Display the correlation table for the entire dataset
corr_styler_list = []
# For some reason, having to force everything to floats
met_data_synth[['Rainfall sum_s', 'Temp mean_s', 'Pressure mean_s', 'Windspeed mean_s', 'Windspeed linregress_s']] = met_data_synth[['Rainfall sum_s', 'Temp mean_s', 'Pressure mean_s', 'Windspeed mean_s',  'Windspeed linregress_s']].astype(float)
df_corr =  get_corr(met_data_synth.drop(columns = ["season"]))    
df_styler = my_styler(df_corr, 2,  "Correlation - all data")
df_styler = styler_highlight_between(df_styler, 0.3, 0.99)
df_styler = styler_highlight_between(df_styler, -0.99, -0.3)
corr_styler_list = [df_styler]

# Get correlation matrices for the 4 seasons and display
for i, item in enumerate(class_names):
    # Extract the data related to one season
    season_data = met_data_synth[met_data_synth["season"] == item].copy()
    # Strip the class column before passing to function
    season_data.drop(columns = ["season"], inplace=True) 
    # For some reason, having to force everything to floats
    season_data[['Rainfall sum_s', 'Temp mean_s', 'Pressure mean_s', 'Windspeed mean_s', 'Windspeed linregress_s']] = season_data[['Rainfall sum_s', 'Temp mean_s', 'Pressure mean_s', 'Windspeed mean_s', 'Windspeed linregress_s']].astype(float)
    df_corr = get_corr(season_data)      
    df_styler = my_styler(df_corr, 2, item + " Correlation")
    df_styler = styler_highlight_between(df_styler, 0.3, 0.99)
    df_styler = styler_highlight_between(df_styler, -0.99, -0.3)
    corr_styler_list.append(df_styler)
    
for item in corr_styler_list:
    display(item)

## Conclusions <a id="Conclusions"></a> <span style="font-size: 8pt;"> [[TOC]](#toc)</span>

- The dataset which comprised hourly meteorological data (2012 - 2022) was aggregated and analysed on the basis of splitting the data into the 4 meteorological seasons.
- There was no strong correlation (>0.7 deemed to be reasonably strong correlation) found between any of the variables in the real dataset (weak correlations were found between pressure and rainfall and pressure and wind speed). The highest correlation value was -0.47 between pressure and rainfall.
- For each of the 4 chosen meteorological variables, a suitable distribution was chosen and fitted to the data for each of the 4 seasons. Parameters for PDF generation were stored for each season and each variable
- Random data was generated for each of the 4 seasons for each data variable for each season.
- To test the random number generation, 1 years worth of data was synthesised with Numpy random distribution generators. The generated histograms were compared to the real dataset.
- Good agreement between real and synthesised data was found for pressure and temperature datasets. Both of these are largely normally distributed. The synthesised wind speed data (Rayleigh) was skewed towards lower values.
- The correlation values are lower than those of the original dataset as no correlation (such as with linear regression or multivariate analysis) was forced on the data.

## References <a id="Refs"></a> <span style="font-size: 8pt;"> [[TOC]](#toc)</span>
1. Met.ie dataset source: https://www.met.ie/climate/available-data/historical-data
2. Replacing blanks with nans: https://stackoverflow.com/questions/13445241/replacing-blank-values-white-space-with-nan-in-pandas
3. Setting/resetting index: https://sparkbyexamples.com/pandas/pandas-set-index-to-column-in-dataframe/
4. Rainfall distribution: https://www.realclimate.org/index.php/archives/2017/11/a-brief-review-of-rainfall-statistics/. 
5. fitting distributions/picking distribution: https://stackoverflow.com/questions/6620471/fitting-empirical-distribution-to-theoretical-ones-with-scipy-python/16651955#16651955
6. Exponential curve fitting:  https://www.geeksforgeeks.org/how-to-do-exponential-and-logarithmic-curve-fitting-in-python/
7. Curve fitting: https://stackoverflow.com/questions/50448199/lognormal-curve-fit
8. Curve fitting: https://www.geeksforgeeks.org/how-to-do-exponential-and-logarithmic-curve-fitting-in-python/
9. Scipy curve fitting: https://stackoverflow.com/questions/2896179/fitting-a-gamma-distribution-with-python-scipy 
10. Scipy manual: https://docs.scipy.org/doc/scipy/reference/stats.html
11. Scipy curve fitting: https://glowingpython.blogspot.com/2012/07/distribution-fitting-with-scipy.html
12. Wind speed PDF: https://www.sciencedirect.com/topics/engineering/wind-speed-distribution
13. Wind speed PDF: https://www.frontiersin.org/articles/10.3389/fenrg.2021.769920/full
14. Numpy generator exponential: https://numpy.org/devdocs/reference/random/generated/numpy.random.Generator.exponential.html
15. Scipy stats details of PDF curve fitting: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.fit.html#scipy.stats.rv_continuous.fit
16. Scipy stats parameters v underlying mu and sigma for lognormal data generation: https://stackoverflow.com/questions/51609299/python-np-lognormal-gives-infinite-results-for-big-average-and-st-dev
17. Multivariate data fit: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.multivariate_normal.html#scipy.stats.multivariate_normal
18. Multivariate ranom data generation: https://numpy.org/doc/stable/reference/random/generated/numpy.random.multivariate_normal.html
19. Linear regression in scipy: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html
